In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, LeaveOneOut, KFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

In [ ]:

data = pd.read_pickle('../create_SL_data/data_6months.pkl')

data

In [ ]:
data['patient_number'].unique()

In [ ]:
X = data.drop(['CHR', 'patient_number'], axis=1)  
y = data['CHR']                                  

patient_numbers = data['patient_number']

In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import LeaveOneOut, GridSearchCV
from sklearn.metrics import f1_score, accuracy_score

models = {
    'RandomForest': {
        'model': RandomForestClassifier(random_state=42),
        'params': {'n_estimators': [50, 100, 200, 300, 500], 'max_depth': [3, 5, 10, 50], 'max_features' : ['sqrt', None]}

    },
    'KNN': {
        'model': KNeighborsClassifier(),
        'params': {'n_neighbors': [3, 5, 7, 10], 'weights': ['uniform', 'distance']}
    },
    'LogisticRegression': {
        'model': LogisticRegression(random_state=42, max_iter=1000),
        'params': {'C': [0.1, 1, 10], 'solver': ['liblinear', 'lbfgs']}
    },
    'LDA': {
        'model': LinearDiscriminantAnalysis(),
        'params': {'solver': ['svd', 'lsqr']}
    }
}

feature_importances = {name: pd.DataFrame(np.zeros((X.shape[0], X.shape[1])), columns=X.columns) for name in models if name == 'RandomForest'}
scores = {name: {'f1_scores': [], 'accuracy_scores': [], 'best_params': []} for name in models}

outer_cv = LeaveOneOut()
inner_cv = LeaveOneOut()

for fold_idx, (train_idx, test_idx) in enumerate(outer_cv.split(X)):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    for name, config in models.items():
        grid_search = GridSearchCV(config['model'], config['params'], cv=inner_cv, scoring='accuracy')
        grid_search.fit(X_train, y_train)
        best_model = grid_search.best_estimator_
        
        y_pred = best_model.predict(X_test)
        f1 = f1_score(y_test, y_pred, average='macro', zero_division=0)
        accuracy = accuracy_score(y_test, y_pred)

        scores[name]['f1_scores'].append(f1)
        scores[name]['accuracy_scores'].append(accuracy)
        scores[name]['best_params'].append(grid_search.best_params_)

        if name == 'RandomForest':
            feature_importances[name].iloc[fold_idx, :] = best_model.feature_importances_
        
    print('fold', fold_idx)

results = {}
for name in models:
    average_f1 = np.mean(scores[name]['f1_scores'])
    average_accuracy = np.mean(scores[name]['accuracy_scores'])
    results[name] = {'Average F1 Score': average_f1, 'Average Accuracy': average_accuracy, 'Best Parameters': scores[name]['best_params'][-1]}

    if name == 'RandomForest':
        mean_fi = feature_importances[name].mean(axis=0).sort_values(ascending=False)
        results[name]['Feature Importances'] = mean_fi

for name, result in results.items():
    print(f"{name} Results:")
    for key, value in result.items():
        print(f"{key}: {value}")


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [ ]:
model1 = RandomForestClassifier(n_estimators=50, max_depth=3, max_features='sqrt', random_state=42)
model2 = KNeighborsClassifier(n_neighbors=3, weights='uniform')
model3 = LinearDiscriminantAnalysis(solver='svd')

model1.fit(X, y)
model2.fit(X, y)
model3.fit(X, y)

In [ ]:
model1.predict(X)

In [ ]:
model2.predict(X)

In [ ]:
model3.predict(X)

In [ ]:
model = RandomForestClassifier(n_estimators=50, max_depth=3, max_features='sqrt', random_state=42)
model.fit(X, y)

In [ ]:
pd.set_option('display.max_colwidth', None)

coefficients = model.feature_importances_  

feature_importance = pd.DataFrame(data={'Feature': X.columns, 'Importance': coefficients})

feature_importance['Absolute Importance'] = feature_importance['Importance'].abs()
feature_importance = feature_importance.sort_values(by='Absolute Importance', ascending=False)

feature_importance.head(30)

In [ ]:
model.score(X, y)

In [ ]:
data_pool2 = pd.read_pickle('../create_SL_data_pool2/data_6months_an_pool2.pkl')
data_pool2

In [ ]:
X_test = data_pool2.drop(['CHR', 'patient_number'], axis=1)  
y_test = data_pool2['CHR']  

y_test    

In [ ]:
pred = model.predict(X_test)

In [ ]:
pred